<a href="https://colab.research.google.com/github/sleman-alterkawi/Data-analysis/blob/Customer-feedback-analysis/Bellabeat_customer_feedback_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import re

# Set random seed for reproducibility
np.random.seed(42)

# --- 1. SETUP: Creating a Sample DataFrame with data exhibiting the desired correlations ---

# Simulating strong negative correlation (Time Since Update vs. Engagement Score)
time_since_update = np.random.uniform(10, 100, 100)
engagement = 100 - (time_since_update * 0.8) + np.random.normal(0, 15, 100)
engagement = np.clip(engagement, 10, 100) # Ensure scores are reasonable

# Simulating moderate positive correlation (Feature A Usage vs. Support Volume)
feature_a_usage = np.random.randint(50, 500, 100)
support_tickets = np.round((feature_a_usage / 100) + np.random.normal(0, 2, 100))
support_tickets = np.clip(support_tickets, 1, 15) # Ensure ticket count is positive

df = pd.DataFrame({
    'Time_Since_Last_Update (days)': time_since_update,
    'Customer_Engagement_Score': engagement,
    'Feature_A_Usage_Count': feature_a_usage,
    'Support_Ticket_Volume': support_tickets.astype(int),
    'Project_Revenue': np.random.randint(5000, 50000, 100),
    'Project_Duration (in days)': np.random.randint(10, 180, 100),
})

# Adding a simulated text column for TF-IDF
feedback_data_base = [
    "The login system is slow and I keep getting an error.",
    "Great product, very intuitive and fast.",
    "I had a billing issue but support was quick.",
    "The new feature is confusing and causes error codes.",
    "Everything is reliable and works perfectly.",
    "My login keeps timing out. Please fix the slow performance.",
    "The documentation is clear and the app is very stable.",
    "Another error, I cannot complete my task because of a billing issue.",
    "Intuitive design, but the login process is cumbersome.",
    "Fast service and reliable features.",
    "Slow slow slow performance and system errors.",
    "Customer support was unhelpful with my complex billing query.",
    "Excellent, intuitive experience.",
    "The login timeout error is a major problem.",
]
feedback_data = feedback_data_base * 8 # Repeat to make a larger sample to cover 100 rows

# Introduce a small number of NaN/missing values to demonstrate robustness
df['Customer_Feedback'] = feedback_data[:100]
df.loc[5, 'Project_Duration (in days)'] = np.nan
df.loc[10, 'Project_Revenue'] = np.nan

print("--- Initial Data Snapshot ---")
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))
print("\n" + "="*80 + "\n")

--- Initial Data Snapshot ---
| Time_Since_Last_Update (days)   | Customer_Engagement_Score   | Feature_A_Usage_Count   | Support_Ticket_Volume   | Project_Revenue   | Project_Duration (in days)   | Customer_Feedback                                     |
|:--------------------------------|:----------------------------|:------------------------|:------------------------|:------------------|:-----------------------------|:------------------------------------------------------|
| 43.7086                         | 66.3388                     | 275                     | 2                       | 42497             | 161                          | The login system is slow and I keep getting an error. |
| 95.5643                         | 19.0635                     | 326                     | 3                       | 14348             | 165                          | Great product, very intuitive and fast.               |
| 75.8795                         | 40.6728                     | 335 

In [10]:
# --- 2. DIVE DEEPER: CORRELATION ANALYSIS (Pearson 'r') ---

print("--- Section 2: Correlation Analysis (Pearson 'r') ---")

# Correlation 1: Strong Negative Relationship
corr_1 = df['Time_Since_Last_Update (days)'].corr(df['Customer_Engagement_Score'])
print(f"1. Time Since Last Update vs. Engagement Score (r): {corr_1:.4f} (Strong Negative)")

# Correlation 2: Moderate Positive Relationship
corr_2 = df['Feature_A_Usage_Count'].corr(df['Support_Ticket_Volume'])
print(f"2. Feature A Usage Count vs. Support Ticket Volume (r): {corr_2:.4f} (Moderate Positive)")

print("\n" + "="*80 + "\n")

--- Section 2: Correlation Analysis (Pearson 'r') ---
1. Time Since Last Update vs. Engagement Score (r): -0.8749 (Strong Negative)
2. Feature A Usage Count vs. Support Ticket Volume (r): 0.4811 (Moderate Positive)




In [11]:
# --- 3. NEW METRICS (Feature Engineering) ---

print("--- Section 3: New Metric Creation ---")

# Ensure we handle potential NaN values by dropping them for the calculation, or fill with 0/mean if appropriate
df_cleaned_metrics = df.dropna(subset=['Project_Revenue', 'Project_Duration (in days)']).copy()

# Metric 1: Value Density (Revenue per unit of Time)
df_cleaned_metrics['Value_Density'] = df_cleaned_metrics['Project_Revenue'] / df_cleaned_metrics['Project_Duration (in days)']
print("Metric 1: Value Density (Project Revenue / Duration)")
print(df_cleaned_metrics[['Project_Revenue', 'Project_Duration (in days)', 'Value_Density']].head().to_markdown(index=False, numalign="left", stralign="left"))

# Metric 2: Adoption-to-Support Ratio
# Adding 1 to the denominator to avoid division by zero errors if ticket volume is 0
df_cleaned_metrics['Adoption_to_Support_Ratio'] = df_cleaned_metrics['Feature_A_Usage_Count'] / (df_cleaned_metrics['Support_Ticket_Volume'] + 1)
print("\nMetric 2: Adoption-to-Support Ratio (Feature Usage / (Tickets + 1))")
print(df_cleaned_metrics[['Feature_A_Usage_Count', 'Support_Ticket_Volume', 'Adoption_to_Support_Ratio']].head().to_markdown(index=False, numalign="left", stralign="left"))

df = df_cleaned_metrics # Use the cleaned dataframe for subsequent steps
print("\n" + "="*80 + "\n")

--- Section 3: New Metric Creation ---
Metric 1: Value Density (Project Revenue / Duration)
| Project_Revenue   | Project_Duration (in days)   | Value_Density   |
|:------------------|:-----------------------------|:----------------|
| 42497             | 161                          | 263.957         |
| 14348             | 165                          | 86.9576         |
| 28714             | 126                          | 227.889         |
| 43102             | 17                           | 2535.41         |
| 13924             | 131                          | 106.29          |

Metric 2: Adoption-to-Support Ratio (Feature Usage / (Tickets + 1))
| Feature_A_Usage_Count   | Support_Ticket_Volume   | Adoption_to_Support_Ratio   |
|:------------------------|:------------------------|:----------------------------|
| 275                     | 2                       | 91.6667                     |
| 326                     | 3                       | 81.5                        |
| 335 

In [12]:
# --- 4. LINEAR REGRESSION SETUP (For future prediction) ---

print("--- Section 4: Linear Regression Setup ---")

# We will use Correlation 1 (Time Since Last Update vs. Engagement Score) since it is strongly linear.

# Define features (X) and target (y)
X = df[['Time_Since_Last_Update (days)']]
y = df['Customer_Engagement_Score']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Output key model parameters
print(f"Independent Variable (X): Time_Since_Last_Update (days)")
print(f"Dependent Variable (y): Customer_Engagement_Score")
print(f"Model Intercept (b0): {model.intercept_:.2f}")
print(f"Model Coefficient (b1): {model.coef_[0]:.2f}")
print(f"R-squared on Test Data (Model Fit): {model.score(X_test, y_test):.2f}")
print("\nPrediction Example (120 days since update):")
prediction = model.predict([[120]])[0]
print(f"Predicted Engagement Score: {prediction:.2f}")

print("\n" + "="*80 + "\n")

--- Section 4: Linear Regression Setup ---
Independent Variable (X): Time_Since_Last_Update (days)
Dependent Variable (y): Customer_Engagement_Score
Model Intercept (b0): 99.13
Model Coefficient (b1): -0.81
R-squared on Test Data (Model Fit): 0.78

Prediction Example (120 days since update):
Predicted Engagement Score: 2.44




/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [13]:
# --- 5. TEXTUAL ANALYSIS (TF-IDF) ---

print("--- Section 5: Textual Analysis (TF-IDF) ---")

# Preprocessing: Remove punctuation and convert to lowercase
def preprocess_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text) # Remove punctuation
    return text

df['Cleaned_Feedback'] = df['Customer_Feedback'].apply(preprocess_text)

# Initialize TF-IDF Vectorizer
# max_features limits the number of features, stop_words filters common words
tfidf = TfidfVectorizer(max_features=20, stop_words='english')

# Fit and transform the cleaned text data
tfidf_matrix = tfidf.fit_transform(df['Cleaned_Feedback'])
feature_names = tfidf.get_feature_names_out()
tfidf_sums = tfidf_matrix.sum(axis=0)

# Create a DataFrame of terms and their summed TF-IDF scores
tfidf_df = pd.DataFrame(tfidf_sums, columns=feature_names).T.rename(columns={0: 'Total_TFIDF_Score'})
tfidf_df = tfidf_df.sort_values(by='Total_TFIDF_Score', ascending=False)

print("Top 10 Key Terms by Total TF-IDF Score (Themes Discovered):")
print(tfidf_df.head(10).to_markdown(numalign="left", stralign="left"))
print("\nInterpretation: High TF-IDF scores indicate terms that are frequent in some documents but not across all, making them highly specific and informative for key themes.")

--- Section 5: Textual Analysis (TF-IDF) ---
Top 10 Key Terms by Total TF-IDF Score (Themes Discovered):
|             | Total_TFIDF_Score   |
|:------------|:--------------------|
| login       | 14.3338             |
| error       | 12.9022             |
| slow        | 12.8142             |
| billing     | 12.2239             |
| reliable    | 12.0054             |
| intuitive   | 11.5222             |
| support     | 9.50746             |
| issue       | 8.80551             |
| fast        | 8.60026             |
| performance | 6.20997             |

Interpretation: High TF-IDF scores indicate terms that are frequent in some documents but not across all, making them highly specific and informative for key themes.
